In [3]:
import numpy
import json

In [5]:
data = json.load(open('annotated.json'))
data

[{'id': 1,
  'annotations': [{'id': 353,
    'completed_by': 1,
    'result': [],
    'was_cancelled': False,
    'ground_truth': False,
    'created_at': '2025-08-29T12:15:15.743623Z',
    'updated_at': '2025-08-29T12:15:15.743655Z',
    'draft_created_at': None,
    'lead_time': 4.565,
    'prediction': {},
    'result_count': 0,
    'unique_id': '6681ffc6-57f6-48b3-896d-c2b22fea5bb6',
    'import_id': None,
    'last_action': None,
    'bulk_created': False,
    'task': 1,
    'project': 4,
    'updated_by': 1,
    'parent_prediction': None,
    'parent_annotation': None,
    'last_created_by': None}],
  'file_upload': 'd65f4761-unlabelled_sentences.json',
  'drafts': [],
  'predictions': [],
  'data': {'text': 'In this issue of Cell Stem Cell, Shen et\xa0al.'},
  'meta': {},
  'created_at': '2025-08-28T11:02:08.895073Z',
  'updated_at': '2025-08-29T12:15:15.786966Z',
  'inner_id': 1,
  'total_annotations': 1,
  'cancelled_annotations': 0,
  'total_predictions': 0,
  'comment_count'

In [ ]:
# print(data[13]['annotations'][0]['result'][0])

text = data[13]['data']['text']

spans = [r for r in data[13]['annotations'][0]['result']]

[{'value': {'start': 0, 'end': 3, 'text': 'Age', 'labels': ['ENTITY']},
  'id': 'sa6IRr9hCt',
  'from_name': 'label',
  'to_name': 'text',
  'type': 'labels',
  'origin': 'manual'},
 {'value': {'start': 4, 'end': 13, 'text': '≥74 years', 'labels': ['VALUE']},
  'id': 't_E3dj0Mup',
  'from_name': 'label',
  'to_name': 'text',
  'type': 'labels',
  'origin': 'manual'},
 {'value': {'start': 241,
   'end': 257,
   'text': 'hemoglobin level',
   'labels': ['ENTITY']},
  'id': 'ZiGOH1w4n8',
  'from_name': 'label',
  'to_name': 'text',
  'type': 'labels',
  'origin': 'manual'},
 {'value': {'start': 258,
   'end': 267,
   'text': '<10 mg/dL',
   'labels': ['VALUE']},
  'id': 'yv2_hydPPd',
  'from_name': 'label',
  'to_name': 'text',
  'type': 'labels',
  'origin': 'manual'},
 {'value': {'start': 269,
   'end': 296,
   'text': 'glomerular filtration rate ',
   'labels': ['ENTITY']},
  'id': 'quKxeHD_zH',
  'from_name': 'label',
  'to_name': 'text',
  'type': 'labels',
  'origin': 'manual'},
 {'

In [41]:
def extract_info(entry):
    text = entry["data"]["text"]
    annotations = entry.get("annotations", [])
    entities, values, relations = [], [], []

    for ann in annotations:
        results = ann.get("result", [])
        id_to_text = {}
        id_to_label = {}

        # First pass: collect entities/values
        for res in results:
            if res["type"] == "labels":
                label_type = res["value"]["labels"][0]
                span_text = res["value"]["text"]
                res_id = res["id"]

                id_to_text[res_id] = span_text
                id_to_label[res_id] = label_type

                if label_type == "ENTITY":
                    entities.append(span_text)
                elif label_type == "VALUE":
                    values.append(span_text)

        # Second pass: collect relations
        for res in results:
            if res["type"] == "relation":
                from_id, to_id = res["from_id"], res["to_id"]
                if from_id in id_to_text and to_id in id_to_text:
                    relations.append(f"{id_to_text[from_id]} : {id_to_text[to_id]}")

    return {
        "text": text,
        "entities": entities,
        "values": values,
        "relations": relations
    }
extracted_data = [extract_info(entry) for entry in data]

In [43]:
extracted_data[13]

{'text': 'Age ≥74 years, male sex, history of carpal tunnel, left ventricular ejection fraction <55%, posterior wall thickness over 12 mm, and relative wall thickness over 0.57 were independent predictors of positive Tc-PYP results in our cohort, and hemoglobin level <10 mg/dL, glomerular filtration rate <30 mL/min/1.73 m Two simple clinical scoring systems, derived to identify patients at high risk of having ATTR-CM necessitating further diagnostic evaluation, showed good predictive accuracy in our diverse patient cohort.',
 'entities': ['Age', 'hemoglobin level', 'glomerular filtration rate '],
 'values': ['≥74 years', '<10 mg/dL', '<30 mL/min/1.73 m'],
 'relations': ['Age : ≥74 years',
  'hemoglobin level : <10 mg/dL',
  'glomerular filtration rate  : <30 mL/min/1.73 m']}

In [44]:
def format_example(example):
    text = example["text"]
    relations = example["relations"]
    # linearize the relations
    rel_string = " ; ".join([rel.replace(":", "@REL@") for rel in relations])
    return {"input": text, "output": rel_string}

formatted_data = [format_example(ex) for ex in extracted_data]

In [46]:
formatted_data[12]

{'input': 'We report a 74-year-old male who presented with progressively drooping shoulders followed by other muscular involvement without other organ involvement as a manifestation of amyloidosis.',
 'output': 'old @REL@ 74-year'}

In [47]:
# save into a json file
with open('formatted_data.json', 'w') as f:
    json.dump(formatted_data, f, indent=4)
    